In [ ]:
# Paso 1 - Reinstalar NumPy (esto fuerza reinicio)
!pip install --upgrade --force-reinstall numpy==1.23.5


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:

# ========================
# 🔥 ELIMINAR BACKENDS CONFLICTIVOS
# ========================
!pip uninstall -y opencv-python opencv-python-headless imageio[opencv] imageio[pyav] av


In [ ]:
# ========================
# 🔧 DEPENDENCIAS DEL SISTEMA
# ========================
!apt-get update && apt-get install -y libglfw3-dev libglew-dev

# ========================
# 🔧 DEPENDENCIAS DE PYTHON
# ========================
!pip install ninja
!pip install gdown
!pip install imageio tifffile
!pip install matplotlib scikit-image lpips plyfile imageio-ffmpeg

In [ ]:

# ========================
# 📁 DESCARGA Y PREPARACIÓN DEL REPO
# ========================
!git clone --recursive https://github.com/caiyuanhao1998/hdr-gs.git
%cd hdr-gs

# Eliminar y reemplazar submódulo roto
!rm -rf /content/hdr-gs/submodules/diff-gaussian-rasterization
!git clone https://github.com/graphdeco-inria/diff-gaussian-rasterization.git /content/hdr-gs/submodules/diff-gaussian-rasterization
!git clone https://github.com/g-truc/glm.git /content/hdr-gs/submodules/diff-gaussian-rasterization/third_party/glm


In [ ]:

# Compilar diff-gaussian-rasterization
%cd /content/hdr-gs/submodules/diff-gaussian-rasterization
import os
os.environ['TORCH_CUDA_ARCH_LIST'] = "7.5;8.6"
!python setup.py build_ext --inplace

# Agregar al path
import sys
sys.path.append('/content/hdr-gs/submodules/diff-gaussian-rasterization')

# Compilar simple-knn
!git clone https://github.com/camenduru/simple-knn.git /content/hdr-gs/submodules/simple-knn
%cd /content/hdr-gs/submodules/simple-knn
!python setup.py build_ext --inplace
sys.path.append('/content/hdr-gs/submodules/simple-knn')


In [ ]:

# ========================
# 📂 DESCARGA Y PREPARACIÓN DEL DATASET
# ========================
%cd /content
!gdown --id 1-9K8_iFwFH3SeTcRoaOTed-h9GBdRrrr -O dataset.zip
!unzip dataset.zip -d dataset
!rm dataset.zip
!mv dataset/data_hdr /content/hdr-gs && rmdir dataset

# ========================
# 🔍 CREAR CARPETA DE PESOS PREENTRENADOS
# ========================
!mkdir -p /content/hdr-gs/pretrained_weights/bathroom


In [ ]:

# ⛔ AQUI FALTA: copiar los pesos a /content/hdr-gs/pretrained_weights/bathroom
# (Subirlos manualmente o usar gdown si están en Drive)


In [ ]:

# ========================
# 🛠️ MODIFICACIÓN NECESARIA EN CÓDIGO
# ========================
# Abrir y modificar estos archivos:
# 1. `scene/dataset_readers.py`
#    Reemplazar la línea:
#       image_hdr_np = np.array(imageio.imread(image_hdr_path)).astype(np.float32)
#    Por:
#       import imageio.v3 as iio
#       image_hdr_np = iio.imread(image_hdr_path).astype(np.float32)

# 2. `train_synthetic.py`
#    Reemplazar guardado con:
#       iio.imwrite(output_path, img_np, extension=".exr")
#    o simplemente:
#       iio.imwrite(output_path, img_np)


In [ ]:

# ========================
# 🚀 EJECUTAR ENTRENAMIENTO
# ========================
%cd /content/hdr-gs

!PYTHONPATH=$PYTHONPATH:/content/hdr-gs/submodules/diff-gaussian-rasterization:/content/hdr-gs/submodules/simple-knn \
python train_synthetic.py \
  --config config/bathroom.yaml \
  --eval \
  --gpu_id 0 \
  --syn
